In [1]:
import numpy as np
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data.csv')

# Print the shape of different sentimental data
print(data[(data['sentiment_score'] > .43) & (data['sentiment_score'] > 0)].shape)
print(data[(data['sentiment_score'] <= .43) & (data['sentiment_score'] > 0)].shape)
print(data[(data['sentiment_score'] > -.53) & (data['sentiment_score'] < 0)].shape)
print(data[(data['sentiment_score'] <= -.53) & (data['sentiment_score'] < 0)].shape)

(66246, 3)
(67601, 3)
(73966, 3)
(70952, 3)


In [3]:
data.head()

,text,sentiment_score,sentiment
0,johnconyersjr alfranken why have you guys no...,0.2575,loosely positive
1,women have been talking about this crap the en...,-0.4588,loosely negative
2,bettemidler please speak to this sexual assaul...,-0.3612,loosely negative
3,we cant keep turning a blind eye and pretend t...,0.2163,loosely positive
4,jayz is saying what ive been saying djts jobis...,-0.6249,strongly negative


In [4]:
print(data[ data['sentiment'] == 'strongly positive'].shape)
print(data[ data['sentiment'] == 'strongly negative'].shape)
print(data[ data['sentiment'] == 'loosely positive'].shape)
print(data[ data['sentiment'] == 'loosely negative'].shape)
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

(66246, 3)
(70952, 3)
(67601, 3)
(73966, 3)


In [5]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 54, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 54, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 788       
Total params: 511,588
Trainable params: 511,588
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.5, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
#print(Y)

(139382, 54) (139382, 4)
(139383, 54) (139383, 4)


In [7]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 40, batch_size=batch_size, verbose = 2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
 - 899s - loss: 0.5898 - acc: 0.7629
Epoch 2/40
 - 854s - loss: 0.4306 - acc: 0.8313
Epoch 3/40
 - 855s - loss: 0.3802 - acc: 0.8534
Epoch 4/40
 - 859s - loss: 0.3516 - acc: 0.8650
Epoch 5/40
 - 855s - loss: 0.3317 - acc: 0.8732
Epoch 6/40
 - 855s - loss: 0.3150 - acc: 0.8802
Epoch 7/40
 - 860s - loss: 0.3024 - acc: 0.8847
Epoch 8/40
 - 856s - loss: 0.2922 - acc: 0.8881
Epoch 9/40
 - 855s - loss: 0.2833 - acc: 0.8927
Epoch 10/40
 - 855s - loss: 0.2737 - acc: 0.8954
Epoch 11/40
 - 856s - loss: 0.2672 - acc: 0.8983
Epoch 12/40
 - 858s - loss: 0.2606 - acc: 0.9009
Epoch 13/40
 - 861s - loss: 0.2579 - acc: 0.9012
Epoch 14/40
 - 858s - loss: 0.2528 - acc: 0.9037
Epoch 15/40
 - 855s - loss: 0.2482 - acc: 0.9056
Epoch 16/40
 - 853s - loss: 0.2443 - acc: 0.9068
Epoch 17/40
 - 855s - loss: 0.2409 - acc: 0.9073
Epoch 18/40
 - 855s - loss: 0.2390 - acc: 0.9089
Epoch 19/40
 - 855s - loss: 0.2379 - acc: 0.9091
Epoch 20/40
 - 855s - loss: 0.

In [8]:
validation_size = 68000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.37
acc: 0.88


In [9]:
#Validation
loosely_neg_cnt,loosely_pos_cnt,strongly_neg_cnt,strongly_pos_cnt,loosely_neg_correct,loosely_pos_correct,strongly_neg_correct,strongly_pos_correct = 0,0,0,0,0,0,0,0

for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            loosely_neg_correct += 1
        elif np.argmax(Y_validate[x]) == 1:
            loosely_pos_correct += 1
        elif np.argmax(Y_validate[x]) == 2:
            strongly_neg_correct += 1
        elif np.argmax(Y_validate[x]) == 3:
            strongly_pos_correct += 1
            
    if np.argmax(Y_validate[x]) == 0:
        loosely_neg_cnt += 1
    elif np.argmax(Y_validate[x]) == 1:
        loosely_pos_cnt += 1
    elif np.argmax(Y_validate[x]) == 2:
        strongly_neg_cnt += 1
    elif np.argmax(Y_validate[x]) == 3:
        strongly_pos_cnt += 1

print("loosely_pos_acc", loosely_pos_correct/loosely_pos_cnt*100, "%")
print("loosely_neg_acc", loosely_neg_correct/loosely_neg_cnt*100, "%")
print("strongly_pos_acc", strongly_pos_correct/strongly_pos_cnt*100, "%")
print("strongly_neg_acc", strongly_neg_correct/strongly_neg_cnt*100, "%")

loosely_pos_acc 86.04124594578056 %
loosely_neg_acc 85.43555921956582 %
strongly_pos_acc 89.64365256124721 %
strongly_neg_acc 90.21965317919076 %


In [18]:
twt = ['It is amazing how beautifully written it is.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=54, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)
print(np.argmax(sentiment))
if np.argmax(sentiment) == 0:
    print("Loosely Negative")
elif np.argmax(sentiment) == 1:
    print("Loosely Positive")
elif np.argmax(sentiment) == 2:
    print("Strongly Negative")
elif np.argmax(sentiment) == 3:
    print("Strongly Positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   22    8  596   66 1809   22    8]]
[1.6951404e-03 4.0258136e-02 1.8217212e-04 9.5786452e-01]
3
Strongly Positive
